In [1]:
import pandas as pd

# Dataset preprocessing

In [2]:
dataset = pd.read_csv("data/adult.data", header=None)

In [3]:
dataset.columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","salary"]

In [4]:
dataset = dataset[(dataset!=" ?").all(axis=1)].reset_index(drop=True)
df_strings = dataset.select_dtypes(['object'])
dataset[df_strings.columns] = df_strings.apply(lambda x: x.str.strip())

In [5]:
clean_dataset = pd.DataFrame(dataset["age"])
for col in dataset.columns[1:-1]:
    if(dataset[col].dtype =='O'):
        clean_dataset = clean_dataset.join(pd.get_dummies(dataset[col], prefix=col))
    else:
        clean_dataset = clean_dataset.join(dataset[col])
labels = (dataset["salary"]==">50K")*1
dataset = clean_dataset

In [8]:
labels

0        0
1        0
2        0
3        0
4        0
        ..
30157    0
30158    1
30159    0
30160    0
30161    1
Name: salary, Length: 30162, dtype: int32

# Knn

# Decision tree

# Cross validation